In [78]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import tweepy
lemmatizer = WordNetLemmatizer()

## Pull 20000 tweets using Twitter API

In [79]:
CONSUMER_KEY='aUS6Xhl7WL4QKDqvaWYqvRMT0'
CONSUMER_SECRET='fG7CE3tYOOdupuytJDPw3zAmQK6GEVkdkJIhFmjBZQ4MZ9osYS'
ACCESS_KEY='1194580360627048449-KaPUysYKgLDj8sZbKyECwtkivv74qC'
ACCESS_SECRET='MBJpcsnhLLn6byIzitJue4rz7jIB9xfFcrTxdorahKcP0'

In [80]:
# auth
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY,ACCESS_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

# set searching key word and total number of tweets
query = 'chatgpt'
max_tweets = 20000
# max_tweets = 1001

tweets = []
last_id = -1
while len(tweets) < max_tweets:
    count = max_tweets - len(tweets)
    try:
        new_tweets = api.search_tweets(q=query, lang='en', count=count, max_id=str(last_id - 1), tweet_mode='extended')
    except tweepy.TweepError as e:
        print("Error:", e)
        break
    if not new_tweets:
        break
        
    #append new records
    tweets.extend(new_tweets)
    last_id = new_tweets[-1].id

#extract specific colmuns
tweet_data = []
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

for tweet in tweets:
    tweet_dict = {}
    specialChars = "!@#$%^&*()_+-=,.:;?|@~`()[]"
    if 'retweeted_status' in tweet._json:
        tweet.full_text = tweet._json['retweeted_status']['full_text']
    else:
        tweet.full_text = tweet.full_text
    for i in specialChars:
        tweet.full_text = tweet.full_text.replace(i,'')
        tweet.full_text = tweet.full_text.lower().replace("\"", '') 
    temp = [word for word in tweet.full_text.split() if not word in stop_words]
    
    # Lemmatize text
    lemmatized_words = [lemmatizer.lemmatize(word) for word in temp]
    temp = ' '.join(lemmatized_words)
    tweet_dict['Text'] = temp.split()
    tweet_dict['User'] = tweet.user.screen_name
    tweet_dict['Created At'] = tweet.created_at
    tweet_data.append(tweet_dict)
    
# save to csv file
df = pd.DataFrame(tweet_data)

df.to_csv('tweets.csv')
df.head()

Rate limit reached. Sleeping for: 236
Rate limit reached. Sleeping for: 806


,Text,User,Created At
0,"[google’s, plan, catch, chatgpt, stuff, ai, ma...",energyredefined,2023-04-10 02:38:50+00:00
1,"[using, chatgpt, create, cert, study, plan, gt...",HeyBigDaddy,2023-04-10 02:38:49+00:00
2,"[kirawontmiss, 2040, dying, operating, table, ...",ellieshorse,2023-04-10 02:38:49+00:00
3,"[ep, chatgpt, https//tco/auxgu4lcwq]",Sarah02839,2023-04-10 02:38:44+00:00
4,"[rwidome, use, chatgpt, back, ai, v, ai, mean,...",Freddy101Ready,2023-04-10 02:38:41+00:00


## Selected 1000 samples from tweets.csv

In [81]:
sample_df = pd.read_csv('tweets.csv')
sample = sample_df.sample(n = 1000)
sample = sample.drop('Unnamed: 0', axis = 1)
sample.to_csv('sample.csv')
sample

,Text,User,Created At
19760,"['chatgpt', 'person', 'https//tco/9of7o2sbq6']",CantGuardSmitty,2023-04-09 16:00:14+00:00
10320,"['maybe', 'coolest', 'productive', 'use', 'cha...",0xMazz,2023-04-09 20:02:44+00:00
5852,"['gpt4', 'chatgpt', 'openai', 'artificialintel...",anthonyhadfiel7,2023-04-09 22:41:09+00:00
4259,"['iotday', 'asked', 'chatgpt', 'top', 'contrib...",stefihane,2023-04-09 23:42:25+00:00
14779,"['100', 'million', 'people', 'use', 'chatgpt',...",feynmanverse,2023-04-09 17:50:13+00:00
...,...,...,...
12950,"['patient', 'saw', 'multiple', 'physician', 'n...",djammurray,2023-04-09 18:46:00+00:00
14285,"['patient', 'saw', 'multiple', 'physician', 'n...",Annawatton,2023-04-09 18:03:47+00:00
10791,"['anyone', 'else', 'scared', '🤔', 'ai', 'opena...",O_B_One_II,2023-04-09 19:47:25+00:00
17672,"['hasnainwriter11', 'noahmorriz', 'chatgpt']",Neetoro1,2023-04-09 16:54:41+00:00
